In [1]:
from pyspark.sql import SparkSession

In [ ]:
# Check decryption
spark = (
    SparkSession.builder.appName("app_name")
    .master("local[4]")
    .config("spark.driver.memory", "8g")
    .config("spark.sql.shuffle.partitions", "8")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .getOrCreate()
)

In [ ]:
delta_table_path = "/Users/emif/Documents/payclip/data-tools/spark-warehouse/mutt_data.db/employee_data"

df = spark.read.format("delta").load(delta_table_path)

df.show()